# Visualizing Objectives



Principles:

1. Objective chains are directed and acyclic.

2. Each node represents a discrete competency cluster.

3. Each edge represents a dependency of one competency cluster on another.

4. Competency clusters should be as coherent as possible.

2. Edge chains should be independent.  I.e. one should not link to more than one previous node in a chain.  (One may link to both nodes in a forked chain.)
    
    For instance, if 303 depends on 170 and 190, but 190 depends on 170, then only 190 should be linked to 303.


##  Load and clean CSV of nodes

In [86]:
# Load and clean CSV of nodes and descriptions.
from csv import DictReader

nodes = {}
edges = {}
with open('curr-raw.csv', 'r') as rawFile:
    rawData = DictReader(rawFile)
    
    # Build nodes.
    for line in rawData:
        if line["Topic"] in ['', 'Aggregate'] or line["Competency Cluster"] in '': continue
        nodes[line['#']] = f'{line["Topic"]}: {line["Competency Cluster"]}'
        
        if line['Antecedents'].strip() == '': continue
        edges[line['#']] = []
        ants = line['Antecedents'].split(',')
        for ante in ants:
            if line["Antecedents"].strip() == '0': continue
            edges[line['#']].append(ante.strip())
            #print(line['#'], edges[line['#']])

##  Convert to graph representation

In [87]:
import pygraphviz as pgv

G = pgv.AGraph(edges,
               strict=True,
               directed=True,
               rankdir="RL",
               ranksep="0.25",
               ordering="in")

G.layout('dot')
G.draw('curr.png')
G.draw('curr.svg')

##  Manually build palette for Hoon School topics

In [88]:
data = '''100
110, 113, 103
115, 120, 125, 175
130, 133, 135
140, 145, 160
150, 155, 165
156, 183
184, 233
163, 180
170, 190, 217'''

colors = {
    0:'#9e0142',
    1:'#d53e4f',
    2:'#f46d43',
    3:'#fdae61',
    4:'#fee08b',
    5:'#e6f598',
    6:'#abdda4',
    7:'#66c2a5',
    8:'#3288bd',
    9:'#5e4fa2',
}
#http://colrd.com/palette/18894/ "Spectral 10" by 
nodes = {}
for idx,row in enumerate(data.split('\n')):
    for val in row.split(','):
        nodes[val.strip()] = colors[idx]

##  Mark SVG nodes by color.

In [100]:
from bs4 import BeautifulSoup
import re

with open('curr.svg') as f:
    svg = f.read()

soup = BeautifulSoup(svg, 'lxml')

svg_tag = soup.find('svg')
svg_grf = svg_tag.find('g')
paths = svg_grf.find_all('g', {'class': 'node'})

for path in paths:
    try:
        title = path.findChild('title')
    except:
        continue
    if title.contents[0] in nodes:
        ellipse = path.findChild('ellipse')
        color = nodes[title.contents[0]]
        ellipse['fill'] = f'{color}'
        if title.contents[0] == '217':
            print('here')
            #style="fill: url(#circles-1) #fff;"
            ellipse['fill'] = f'url(#diagonalHatch) {color}';
            pass

with open('curr-marked.svg', 'w') as f:
    svg = '''<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">'''
    color = colors[9]
    svg_str = str(svg_tag)
    idx = svg_str.find('<g')
    svg_hdr = svg_str[:idx]
    svg_defs = f'''<defs><pattern id="diagonalHatch" width="4" height="8" patternTransform="rotate(-45 2 2)" patternUnits="userSpaceOnUse"><path d="M -1,2 l 6,0" stroke="{color}" stroke-width="4"/></pattern></defs>'''
    svg_str = svg+'\n'+svg_hdr+svg_defs+'\n'+svg_str[idx:]
    f.write(svg_str)
    
from cairosvg import svg2png
svg2png(bytestring=(svg+str(svg_tag)), dpi=100, write_to='curr-marked.png')

here


##  Build objectives list.

In [5]:
data = '''100
110, 113, 103
115, 120, 125, 175
130, 133, 135
140, 145, 160
150, 155, 165
156, 183
184, 233
163, 180
170, 190, 217'''

lessons = {}
for idx,row in enumerate(data.split('\n')):
    lessons[idx] = []
    for val in row.split(','):
        lessons[idx].append(val.strip())

import yaml
for lesson in lessons:
    print(f'After Lesson {lesson}, you should be able to:\n')
    for node in lessons[lesson]:
        # whatever your local path is
        fname = f'/home/neal/urbit/curriculum/lessons/{node}.yml'
        with open(fname, 'r') as f:
            data = f.read()
        data = yaml.safe_load(data)
        for obj in data['objectives']:
            print('-', obj)
    
    print('\nYou will know the runes:\n')
    for node in lessons[lesson]:
        # whatever your local path is
        fname = f'/home/neal/urbit/curriculum/lessons/{node}.yml'
        with open(fname, 'r') as f:
            data = f.read()
        data = yaml.safe_load(data)
            
        for rune in data['runes']:
            print(f'- `{rune}`')
    print('\n---\n')

After Lesson 0, you should be able to:

- Explain what an Urbit ship is.
- Identify the `.urb/` directory.
- Distinguish a fakeship from a liveship.
- Use the `+ls` generator to show a directory's contents.
- `|mount` and `|commit` a desk.

You will know the runes:


---

After Lesson 1, you should be able to:

- Distinguish nouns, cells, and atoms.
- Apply auras to transform an atom.
- Identify common Hoon molds, such as cells, lists, and tapes.
- Annotate Hoon code with comments.
- Pin a face to the subject.
- Make a decision at a branch point.
- Distinguish loobean from boolean operations.
- Slam a gate (call a function).
- Produce a generator to convert a value between auras.
- Pronounce ASCII characters per standard Hoon developer practice.

You will know the runes:

- `::`
- `%-`
- `=/`
- `?:`
- `^-`
- `~&`
- ``

---

After Lesson 2, you should be able to:

- Identify current known irregular syntax.
- Convert between regular and irregular forms of runes to date.
- Identify a mold

In [6]:
lessons

{0: ['100'],
 1: ['110', '113', '103'],
 2: ['115', '120', '125', '175'],
 3: ['130', '133', '135'],
 4: ['140', '145', '160'],
 5: ['150', '155', '165'],
 6: ['156', '183'],
 7: ['184', '233'],
 8: ['163', '180'],
 9: ['170', '190', '217']}

In [121]:
topics = '''Aggregate
Ames
API
Arvo
%ask generators
Azimuth
Behn
Clay
CLI
Dill
Distribution
Eyre
Gall
Gall agents
Gall (Spider)
Hoon
Iris
Jael
Khan
Nock
Runtime
Sail
Stdlib
Sysadmin
Testing'''.split('\n')

# Load and clean CSV of nodes and descriptions.
from csv import DictReader

nodes = {}
edges = {}
topicd = {}
with open('curr-raw.csv', 'r') as rawFile:
    rawData = DictReader(rawFile)
    
    # Build nodes.
    for line in rawData:
        if line["Topic"] in ['', 'Aggregate'] or line["Competency Cluster"] in '': continue
        nodes[line['#']] = f'{line["Topic"]}: {line["Competency Cluster"]}'
        
        if line['Antecedents'].strip() == '': continue
        edges[line['#']] = []
        ants = line['Antecedents'].split(',')
        for ante in ants:
            if line["Antecedents"].strip() == '0': continue
            edges[line['#']].append(ante.strip())
            #print(line['#'], edges[line['#']])
        
        if line["Topic"] in topics:
            topic = line['Topic']
            if topic == 'Distribution':  topic = 'Clay'
            if 'Gall' in topic:  topic = 'Gall'
            if 'API' in topic:  topic = 'Stdlib'
            if 'CLI' in topic:  topic = 'Khan'
            if '%ask' in topic:  topic = 'Hoon'
            if 'Sail' in topic:  topic = 'Hoon'
            topicd[line["#"]] = topic

##  Mark SVG nodes by topic

In [123]:
# Hoon/Stdlib/API.CLI are solid color
# Arvo+vanes are striped

colors = [
    '#9e0142',
    '#d53e4f',
    '#f46d43',
    '#fdae61',
    '#fee08b',
    '#e6f598',
    '#abdda4',
    '#66c2a5',
    '#3288bd',
    '#5e4fa2',
]

from numpy.random import choice
colors = {
    'Ames':choice(colors),
    'Behn':choice(colors),
    'Clay':choice(colors),
    'Dill':choice(colors),
    'Eyre':choice(colors),
    'Gall':choice(colors),
    'Iris':choice(colors),
    'Jael':choice(colors),
    'Khan':choice(colors),
    'Arvo':choice(colors),
    'Hoon':choice(colors),
    'Azimuth':choice(colors),
    'Nock':'#ffffff',
    'Runtime':choice(colors),
    'Stdlib':choice(colors),
    'Sysadmin':choice(colors),
    'Testing':choice(colors),
}
hatched = ['Ames', 'Behn', 'Clay', 'Dill', 'Eyre', 'Gall', 'Iris', 'Jael', 'Khan', 'Arvo']

svg_defs = ''
for h in hatched:
    svg_defs += f'''<defs><pattern id="diagonalHatch{h}" width="4" height="8" patternTransform="rotate(-45 2 2)" patternUnits="userSpaceOnUse"><path d="M -1,2 l 6,0" stroke="{colors[h]}" stroke-width="4"/></pattern></defs>'''


from bs4 import BeautifulSoup
import re

with open('curr.svg') as f:
    svg = f.read()

soup = BeautifulSoup(svg, 'lxml')

svg_tag = soup.find('svg')
svg_grf = svg_tag.find('g')
paths = svg_grf.find_all('g', {'class': 'node'})

for path in paths:
    try:
        title = path.findChild('title')
    except:
        continue
    if title.contents[0] in topicd.keys():
        ellipse = path.findChild('ellipse')
        color = colors[topicd[title.contents[0]]]
        ellipse['fill'] = f'{color}'
        if topicd[title.contents[0]] in hatched:
            ellipse['fill'] = f'url(#diagonalHatch{topicd[title.contents[0]]}) {color}';

with open('curr-topics.svg', 'w') as f:
    svg = '''<?xml version="1.0" encoding="UTF-8" standalone="no"?><!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">'''
    color = colors['Testing']
    svg_str = str(svg_tag)
    idx = svg_str.find('<g')
    svg_hdr = svg_str[:idx]
    svg_str = svg+'\n'+svg_hdr+svg_defs+'\n'+svg_str[idx:]
    f.write(svg_str)
    
from cairosvg import svg2png
svg2png(bytestring=(svg+str(svg_tag)), dpi=100, write_to='curr-topics.png')

print(colors)

{'Ames': '#e6f598', 'Behn': '#e6f598', 'Clay': '#abdda4', 'Dill': '#d53e4f', 'Eyre': '#f46d43', 'Gall': '#abdda4', 'Iris': '#d53e4f', 'Jael': '#3288bd', 'Khan': '#5e4fa2', 'Arvo': '#3288bd', 'Hoon': '#5e4fa2', 'Azimuth': '#3288bd', 'Nock': '#ffffff', 'Runtime': '#abdda4', 'Stdlib': '#66c2a5', 'Sysadmin': '#9e0142', 'Testing': '#66c2a5'}
